<a href="https://colab.research.google.com/github/choheeee22/pytorch_study/blob/main/leaf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#구글 드라이브 마운트 

In [1]:
cd /content/drive/MyDrive/Python 공부/leaf

/content/drive/MyDrive/Python 공부/leaf


In [2]:
pwd

'/content/drive/MyDrive/Python 공부/leaf'

In [3]:
ls

fmix.py  __pycache__/


In [4]:
#pip install kaggle --upgrade

In [5]:
 cd /content/drive/MyDrive/Python 공부/kaggle

/content/drive/MyDrive/Python 공부/kaggle


In [6]:
ls

cassava-leaf-disease-classification.zip  test_tfrecords/
kaggle.json                              train.csv
label_num_to_disease_map.json            train_images/
sample_submission.csv                    train_tfrecords/
test_images/


In [7]:
ls -1ha kaggle.json # kaggle.json잘 들어있는지 확인하기 

kaggle.json


In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
#!kaggle competitions list

In [9]:
#데이터 다운
#!kaggle competitions download -c cassava-leaf-disease-classification

In [10]:
ls

cassava-leaf-disease-classification.zip  test_tfrecords/
kaggle.json                              train.csv
label_num_to_disease_map.json            train_images/
sample_submission.csv                    train_tfrecords/
test_images/


In [11]:
#!unzip cassava-leaf-disease-classification.zip

In [12]:
ls

cassava-leaf-disease-classification.zip  test_tfrecords/
kaggle.json                              train.csv
label_num_to_disease_map.json            train_images/
sample_submission.csv                    train_tfrecords/
test_images/


In [18]:
#필요한 라이브러리 다운
import numpy as np 
import pandas as pd
import json
from PIL import Image #이미지 읽고 쓰기 (string 이랑 변환도 하고)
import os

import torch
import torch.nn as nn # torch 신경망 생성
import torchvision
import torchvision.transforms as transforms # 이미지 모양 바꿔주는 거. ex) crop, rotation
from torch.utils.data import Dataset, DataLoader # get_item, len사용할 수 있음 

In [14]:
BATCH = 32
EPOCHS = 20

LR = 0.0001
IM_SIZE = 256

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
TRAIN_DIR = '/content/drive/MyDrive/Python 공부/kaggle/train_images/'
TEST_DIR = '/content/drive/MyDrive/Python 공부/kaggle/test_images/'

In [16]:
pwd

'/content/drive/MyDrive/Python 공부/kaggle'

In [17]:
labels = json.load(open("/content/drive/MyDrive/Python 공부/kaggle/label_num_to_disease_map.json"))
print(labels)

train = pd.read_csv('/content/drive/MyDrive/Python 공부/kaggle/train.csv')
train.head()

{'0': 'Cassava Bacterial Blight (CBB)', '1': 'Cassava Brown Streak Disease (CBSD)', '2': 'Cassava Green Mottle (CGM)', '3': 'Cassava Mosaic Disease (CMD)', '4': 'Healthy'}


,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [19]:
X_Train, Y_Train = train['image_id'].values, train['label'].values
X_Test = [name for name in (os.listdir(TEST_DIR))] # os.listdir() - 지정한 디렉토리 내 모든 파일과 디렉토리를 리스트업 함 

In [20]:
class GetData(Dataset):
  """
  custom dataset 만들기 
  https://wikidocs.net/57165
  """
    def __init__(self, Dir, FNames, Labels, Transform): # 데이터셋의 전처리를 해주는 부분
        self.dir = Dir
        self.fnames = FNames
        self.transform = Transform
        self.lbs = Labels
        
    def __len__(self): # 데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분
        return len(self.fnames)

    def __getitem__(self, index): # class 내 instance 를 slicing 한다 , 데이터셋에서 특정 1개의 샘플을 가져오는 함수
        x = Image.open(os.path.join(self.dir, self.fnames[index]))
        if "train" in self.dir:            
            return self.transform(x), self.lbs[index]            
        elif "test" in self.dir:            
            return self.transform(x), self.fnames[index]

In [21]:
Transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((IM_SIZE, IM_SIZE)),
     transforms.RandomRotation(90),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [22]:
trainset = GetData(TRAIN_DIR, X_Train, Y_Train, Transform)
trainloader = DataLoader(trainset, batch_size=BATCH, shuffle=True, num_workers=4)

testset = GetData(TEST_DIR, X_Test, None, Transform)
testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)

In [31]:
X_images, X_labels = trainset.getitem()

AttributeError: ignored

In [25]:
import matplotlib.pyplot as plt
from torchvision import utils

In [27]:
# display sample images 
def show(img, y=None, color=True):
    npimg = img.numpy()
    npimg_tr = np.transpose(npimg, (1,2,0))
    plt.imshow(npimg_tr)

    if y is not None:
        plt.title('labels :' + str(y))

np.random.seed(1)
torch.manual_seed(1)

grid_size = 4
rnd_inds = np.random.randint(0, len(trainloader), grid_size)
print('image indices:',rnd_inds)

x_grid = [trainloader[i][0] for i in rnd_inds]
y_grid = [trainloader[i][1] for i in rnd_inds]

x_grid = utils.make_grid(x_grid, nrow=grid_size, padding=2)

show(x_grid, y_grid)

image indices: [ 37 235  72 645]


TypeError: ignored

In [ ]:
#trainset 속 value 뽑아보기 
x, y = next(iter(trainset))
print("tranform =", x)
print("\fname =", y)
print("\nsizes of x =")
#for key, value in x.items():
#   print(f"\t{key} = {value.size()}")

tranform = tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.804

In [ ]:
x, y = next(iter(trainloader))
print("tranform =", x)
print("\fname =", y)
print("\nsizes of x =")
#for key, value in x.items():
#   print(f"\t{key} = {value.size()}")

tranform = tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044,

In [ ]:
model = torchvision.models.resnet152() #residual learing, to learn deep layers in better way 
model.fc = nn.Linear(2048, 5, bias=True)

In [ ]:
model = model.to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
for epoch in range(EPOCHS):
    tr_loss = 0.0

    model = model.train()

    for i, (images, labels) in enumerate(trainloader):
        
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        logits = model(images)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss += loss.detach().item()
    
    model.eval()
    print('Epoch: %d | Loss: %.4f'%(epoch, tr_loss / i))

FileNotFoundError: ignored

In [ ]:
s_ls = []

with torch.no_grad():
    model.eval()
    for image, fname in testloader: 
        image = image.to(DEVICE)
        
        logits = model(image)        
        ps = torch.exp(logits)        
        _, top_class = ps.topk(1, dim=1)
        
        for pred in top_class:
            s_ls.append([fname[0], pred.item()])

In [ ]:
sub = pd.DataFrame.from_records(s_ls, columns=['image_id', 'label'])
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)